In [ ]:
from google.colab import files
uploaded= files.upload()

Saving final_burn_dataset_for_colab.zip to final_burn_dataset_for_colab.zip


In [ ]:
import zipfile
import os

zip_path = '/content/final_burn_dataset_for_colab.zip'
output_dir = '/content/' # התיקייה שבה הנתונים יחולצו

print(f"פותח את קובץ ה-ZIP: {zip_path}...")
with zipfile.ZipFile(zip_path, 'r') as zip_ref:
    zip_ref.extractall(output_dir)

print("הנתונים נחלצו בהצלחה ל- /content/final_burn_dataset/")
# לוודא שהנתיבים ש-Colab יצר נכונים
if os.path.exists(os.path.join(output_dir, "final_burn_dataset", "train", "train_images")):
    print("מבנה התיקיות תקין.")
else:
    print("אזהרה: מבנה התיקיות שונה מהצפוי. אנא וודאי את הנתיבים.")

פותח את קובץ ה-ZIP: /content/final_burn_dataset_for_colab.zip...
הנתונים נחלצו בהצלחה ל- /content/final_burn_dataset/
מבנה התיקיות תקין.


In [ ]:
!pip install --upgrade torch torchvision torchaudio --index-url https://download.pytorch.org/whl/cu121
!pip install lxml  # לוודא ש-lxml מותקן לקריאת XMLs

import torch
import torchvision
print(f"Torch version: {torch.__version__}")
print(f"Torchvision version: {torchvision.__version__}")

Looking in indexes: https://download.pytorch.org/whl/cu121
INFO: pip is looking at multiple versions of torch to determine which version is compatible with other requirements. This could take a while.
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 780.5/780.5 MB 2.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 23.7/23.7 MB 100.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 823.6/823.6 kB 60.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 14.1/14.1 MB 125.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 664.8/664.8 MB ? eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 410.6/410.6 MB 1.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 121.6/121.6 MB 7.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 56.5/56.5 MB 13.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 124.2/124.2 MB 7.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 196.0/196.0 MB 6.1 MB/s eta 0:0

In [ ]:
# import torch
# import torchvision
from torchvision.models.detection.faster_rcnn import FasterRCNN_ResNet50_FPN_Weights
from torchvision.models.detection import FasterRCNN
from torch.utils.data import DataLoader, Dataset
from torch.optim.lr_scheduler import ReduceLROnPlateau
import os
import xml.etree.ElementTree as ET
from PIL import Image, ImageDraw, ImageFont
import numpy as np
import warnings
from torchvision import transforms as T

# ייבוא עבור הערכת mAP
from torchvision.models.detection.coco_utils import get_coco_api_from_dataset
from torchvision.models.detection.coco_eval import CocoEvaluator

# הסתרת אזהרות מיותרות מ-Pillow ו-UserWarning כלליות
warnings.filterwarnings("ignore", category=UserWarning)

# --- 1. הגדרות גלובליות ---
ROOT_DIR_TRAIN_IMAGES      = r"/content/final_burn_dataset/train/train_images"
ROOT_DIR_TRAIN_ANNOTATIONS = r"/content/final_burn_dataset/train/train_xmls"
ROOT_DIR_TEST_IMAGES       = r"/content/final_burn_dataset/test/test_images"
ROOT_DIR_TEST_ANNOTATIONS  = r"/content/final_burn_dataset/test/test_xmls"  # ישמש כסט ולידציה

DEVICE = torch.device('cuda') if torch.cuda.is_available() else torch.device('cpu')
print(f"משתמש ב- DEVICE: {DEVICE}")

CLASS_MAP = {
    '__background__': 0, # חובה עבור Faster R-CNN
    'degree_1':       1,
    'degree_2':       2,
    'degree_3':       3
}
NUM_CLASSES = len(CLASS_MAP)  # = 4

# --- 2. פונקציות עזר לטיפול בנתונים ---

def get_transform(train):
    if train:
        return T.Compose([
            T.ToTensor(),
            T.RandomHorizontalFlip(0.5),
            T.ColorJitter(brightness=0.2, contrast=0.2, saturation=0.2, hue=0.02),
            T.RandomAdjustSharpness(sharpness_factor=2, p=0.3),
            T.RandomRotation(degrees=10, expand=False, fill=0), # סיבובים קלים. fill=0 לשחור, expand=False לא משנה גודל תמונה.
        ])
    else: # עבור ולידציה והסקה, רק המרה ל-Tensor
        return T.Compose([ T.ToTensor() ])

def collate_fn(batch):
    """
    פונקציית עזר ל-DataLoader שמשלבת דוגמאות בודדות לבאצ'ים.
    זה נחוץ כאשר גודל התמונות או מספר האובייקטים בהן משתנה.
    """
    return tuple(zip(*batch))

# --- 3. CustomDataset לטעינת נתונים ---
class CustomDataset(Dataset):
    def __init__(self, root_dir_images, root_dir_annotations, transforms=None, class_map=None):
        self.root_dir        = root_dir_images
        self.annotations_dir = root_dir_annotations
        self.transforms      = transforms
        self.class_map       = class_map or CLASS_MAP

        self.image_files_with_xmls = self._load_and_validate_image_xml_pairs()
        # image_ids_int ישמש כ-image_id עבור COCOEvaluator
        # זה חשוב שיהיה ייחודי ורציף (לרוב)
        self.coco_image_ids = list(range(len(self.image_files_with_xmls)))

        print(f"בתיקייה {root_dir_images}: סה\"כ תמונות וביאורים נטענו בהצלחה: {len(self.image_files_with_xmls)}")

    def _load_and_validate_image_xml_pairs(self):
        """
        טוען את כל זוגות קבצי התמונה וה-XML התואמים.
        """
        valid_pairs = []
        img_extensions = ['.jpg', '.jpeg', '.png', '.bmp', '.tiff']

        xml_files = [f for f in os.listdir(self.annotations_dir) if f.endswith('.xml')]
        xml_files.sort() # לוודא סדר קבוע ושחזור עקבי

        for xml_file_name in xml_files:
            base_name = os.path.splitext(xml_file_name)[0]

            found_img_path = None
            for ext in img_extensions:
                temp_path = os.path.join(self.root_dir, base_name + ext)
                if os.path.exists(temp_path):
                    found_img_path = temp_path
                    break

            if found_img_path is None:
                print(f"אזהרה: תמונה {base_name} לא נמצאה עבור ביאור {xml_file_name}. מדלג על זוג זה.")
                continue

            valid_pairs.append((found_img_path, os.path.join(self.annotations_dir, xml_file_name)))
        return valid_pairs

    def __getitem__(self, idx):
        img_path, xml_path = self.image_files_with_xmls[idx]
        current_coco_image_id = self.coco_image_ids[idx] # ה-ID הייחודי עבור COCOEvaluator

        img = Image.open(img_path).convert("RGB")
        width, height = img.size # קבלת מימדי התמונה המקורית

        tree = ET.parse(xml_path)
        root = tree.getroot()

        boxes  = []
        labels = []
        for obj in root.findall('object'):
            name = obj.find('name').text
            if name not in self.class_map:
                print(f"אזהרה: מחלקה '{name}' לא ממופה ב-CLASS_MAP עבור תמונה {img_path}. מדלג על אובייקט זה.")
                continue

            label_id = self.class_map[name]
            bndbox   = obj.find('bndbox')

            # קריאת קואורדינטות ואימותן
            xmin = float(bndbox.find('xmin').text)
            ymin = float(bndbox.find('ymin').text)
            xmax = float(bndbox.find('xmax').text)
            ymax = float(bndbox.find('ymax').text)

            # וודא שהקואורדינטות תקינות (xmin < xmax, ymin < ymax)
            # ושהן בטווח מימדי התמונה. קואורדינטות חייבות להיות חיוביות.
            xmin = max(0.0, xmin)
            ymin = max(0.0, ymin)
            xmax = min(width, xmax)
            ymax = min(height, ymax)

            # אם התיבה לא תקינה לאחר התיקון, מדלגים עליה
            if xmin >= xmax or ymin >= ymax:
                print(f"אזהרה: תיבה לא תקינה ({xmin},{ymin},{xmax},{ymax}) בתמונה {img_path}. מדלג.")
                continue

            boxes.append([xmin, ymin, xmax, ymax])
            labels.append(label_id)

        if len(boxes) == 0:
            # אם אין אובייקטים, יוצרים טנסורים ריקים המתאימים לפורמט
            boxes  = torch.zeros((0, 4), dtype=torch.float32)
            labels = torch.zeros((0,), dtype=torch.int64)
            area   = torch.zeros((0,), dtype=torch.float32)
        else:
            boxes  = torch.as_tensor(boxes, dtype=torch.float32)
            labels = torch.as_tensor(labels, dtype=torch.int64)
            area   = (boxes[:, 3] - boxes[:, 1]) * (boxes[:, 2] - boxes[:, 0])

        target = {
            "boxes":    boxes,
            "labels":   labels,
            "image_id": torch.tensor([current_coco_image_id]), # ה-ID הייחודי עבור COCOEvaluator
            "area":     area,
            "iscrowd":  torch.zeros((len(boxes),), dtype=torch.int64) # כולם לא 'crowd'
        }

        if self.transforms is not None:
            # הטרנספורמציות מוחלות על התמונה.
            # Faster R-CNN ב-torchvision מטפל באופן פנימי בשינויי קואורדינטות התיבות
            # כתוצאה מטרנספורמציות כמו ToTensor.
            img = self.transforms(img)

        return img, target

    def __len__(self):
        return len(self.image_files_with_xmls)

# --- 4. הגדרת המודל ---
def create_fasterrcnn_model(num_classes):
    """
    יוצר מודל Faster R-CNN מאומן מראש על COCO.
    ומתאים אותו למספר המחלקות הספציפי שלנו.
    """
    model = torchvision.models.detection.fasterrcnn_resnet50_fpn(
        weights=FasterRCNN_ResNet50_FPN_Weights.COCO_V1
    )

    # מחליף את ראש החיזוי של המודל כדי להתאים למספר המחלקות החדש שלנו
    in_features = model.roi_heads.box_predictor.cls_score.in_features
    model.roi_heads.box_predictor = torchvision.models.detection.faster_rcnn.FastRCNNPredictor(
        in_features, num_classes
    )
    return model

# --- 5. פונקציות אימון והערכה ---
def train_one_epoch(model, optimizer, data_loader, device, epoch):
    """
    מבצע אימון עבור אפוק אחד.
    """
    model.train() # מעביר את המודל למצב אימון (מאפשר עדכון משקלים)
    total_loss = 0
    for i, (images, targets) in enumerate(data_loader):
        images  = list(image.to(device) for image in images)
        targets = [{k: v.to(device) for k, v in t.items()} for t in targets]

        loss_dict = model(images, targets) # המודל מחזיר מילון הפסדים
        losses    = sum(loss for loss in loss_dict.values()) # סוכם את כל סוגי ההפסדים

        optimizer.zero_grad() # מאפס את הגרדיאנטים
        losses.backward()     # מחשב גרדיאנטים אחורה
        optimizer.step()      # מעדכן את משקלי המודל

        total_loss += losses.item()
        if i % 10 == 0: # מדפיס התקדמות כל 10 איטרציות
            loss_strings = ", ".join([f"{k}: {v.item():.4f}" for k, v in loss_dict.items()])
            print(f"Epoch: {epoch}, Iteration: {i}/{len(data_loader)}, Total Loss: {losses.item():.4f} ({loss_strings})")
    return total_loss / len(data_loader) # ממוצע ההפסד לאפוק

def evaluate(model, data_loader, device, coco_evaluator):
    """
    מבצע הערכה על סט הוולידציה ומחשב מדדי mAP באמצעות COCOEvaluator.
    """
    model.eval() # המודל חייב להיות במצב הערכה כדי לקבל תחזיות (לא לעדכן משקלים)
    print("מבצע הערכה על סט הבדיקה (mAP)...")

    coco_evaluator.reset() # איפוס ה-evaluator לפני כל הערכה חדשה

    with torch.no_grad(): # אין צורך לחשב גרדיאנטים בהערכה
        for i, (images, targets) in enumerate(data_loader):
            images  = list(image.to(device) for image in images)

            # targets חייבים להיות ב-CPU עבור COCOEvaluator
            targets_cpu = [{k: v.cpu() for k, v in t.items()} for t in targets]

            outputs = model(images) # המודל במצב eval מחזיר תחזיות

            # מעבירים את התחזיות ל-CPU עבור COCOEvaluator
            outputs = [{k: v.cpu() for k, v in t.items()} for t in outputs]

            # התאמת הפורמט ל-COCOEvaluator
            # הוא מצפה למילון בו המפתח הוא image_id (מספר שלם) והערך הוא מילון התחזיות
            # השתמש ב-target["image_id"].item() כדי לוודא קבלת ערך סקלרי שלם
            res = {target["image_id"].item(): output for target, output in zip(targets_cpu, outputs)}
            coco_evaluator.update(res)

            if i % 50 == 0: # מדפיס התקדמות כל 50 באצ'ים
                print(f"Eval Batch {i}/{len(data_loader)}")

    # צבירת תוצאות מכל התמונות וסיכום
    coco_evaluator.accumulate()
    coco_evaluator.summarize()

    # ה-mAP הכללי (AP@[IoU=0.50:0.05:0.95|area=all|maxDets=100]) הוא המדד הראשון (אינדקס 0)
    # בטבלת הסטטיסטיקות של COCO. זהו ה-mAP הממוצע על פני ספי IoU רבים.
    overall_mAP = coco_evaluator.coco_eval['bbox'].stats[0]
    return overall_mAP

# --- 6. מחלקת EarlyStopping ---
class EarlyStopping:
    def __init__(self, patience=5, min_delta=0, mode='min'):
        self.patience    = patience    # מספר אפוקס לחכות לפני עצירה
        self.min_delta   = min_delta   # שינוי מינימלי במדד כדי שייחשב שיפור
        self.counter     = 0           # מונה אפוקס ללא שיפור
        self.best_metric = None        # המדד הטוב ביותר שנצפה עד כה
        self.early_stop  = False       # דגל לעצירה מוקדמת
        self.mode        = mode        # 'min' (עבור Loss) או 'max' (עבור mAP)

        if self.mode not in ['min', 'max']:
            raise ValueError("mode חייב להיות 'min' או 'max'")

    def step(self, current_metric):
        if self.best_metric is None: # אם זו הפעם הראשונה
            self.best_metric = current_metric
            return False

        if self.mode == 'min': # למדדים שככל שפחות טוב יותר (כמו Loss)
            if current_metric < self.best_metric - self.min_delta:
                print(f"מדד השתפר מ-{self.best_metric:.4f} ל-{current_metric:.4f}. איפוס מונה.")
                self.best_metric = current_metric
                self.counter     = 0
                return False
            else:
                self.counter += 1
                print(f"EarlyStopping counter: {self.counter} מתוך {self.patience} (מדד לא השתפר מ-{self.best_metric:.4f})")
                if self.counter >= self.patience:
                    self.early_stop = True
                    return True
                return False
        elif self.mode == 'max': # למדדים שככל שיותר טוב יותר (כמו mAP)
            if current_metric > self.best_metric + self.min_delta:
                print(f"מדד השתפר מ-{self.best_metric:.4f} ל-{current_metric:.4f}. איפוס מונה.")
                self.best_metric = current_metric
                self.counter     = 0
                return False
            else:
                self.counter += 1
                print(f"EarlyStopping counter: {self.counter} מתוך {self.patience} (מדד לא השתפר מ-{self.best_metric:.4f})")
                if self.counter >= self.patience:
                    self.early_stop = True
                    return True
                return False
        return False # Fallback במקרה לא צפוי

# --- 7. פונקציית MAIN להרצת תהליך האימון ובהסקה ---
def main():
    # טעינת נתוני אימון
    print("טוען נתוני אימון...")
    dataset_train = CustomDataset(
        root_dir_images=ROOT_DIR_TRAIN_IMAGES,
        root_dir_annotations=ROOT_DIR_TRAIN_ANNOTATIONS,
        transforms=get_transform(train=True),
        class_map=CLASS_MAP
    )

    # טעינת נתוני ולידציה (ה-test directory שלנו)
    print("טוען נתוני בדיקה (ולידציה)...")
    dataset_val = CustomDataset(
        root_dir_images=ROOT_DIR_TEST_IMAGES,
        root_dir_annotations=ROOT_DIR_TEST_ANNOTATIONS,
        transforms=get_transform(train=False),
        class_map=CLASS_MAP
    )

    # יצירת DataLoaders
    data_loader_train = DataLoader(
        dataset_train, batch_size=4, shuffle=True,
        num_workers=2, collate_fn=collate_fn
    )
    data_loader_val = DataLoader(
        dataset_val, batch_size=4, shuffle=False,
        num_workers=2, collate_fn=collate_fn
    )

    # הכנת כלי הערכת COCO לחישוב mAP
    # `get_coco_api_from_dataset` יוצר אובייקט COCO-style מתוך ה-Dataset
    # הוא ישתמש ב-image_id שהוגדר ב-CustomDataset
    coco_dataset_val = get_coco_api_from_dataset(data_loader_val.dataset)
    iou_types = ["bbox"] # עבור זיהוי אובייקטים אנו מעוניינים ב-Bounding Box IOU
    coco_evaluator = CocoEvaluator(coco_dataset_val, iou_types)


    # יצירת המודל והעברתו ל-DEVICE
    print("יוצר מודל Faster R-CNN...")
    model = create_fasterrcnn_model(NUM_CLASSES)
    model.to(DEVICE)

    # הגדרת אופטימייזר ולומד קצב (AdamW ו-ReduceLROnPlateau)
    params    = [p for p in model.parameters() if p.requires_grad]
    optimizer = torch.optim.AdamW(params, lr=0.001, weight_decay=0.0005) # AdamW עם קצב למידה ו-weight_decay

    # ReduceLROnPlateau מוריד את קצב הלמידה כשמדד הוולידציה (mAP) מפסיק להשתפר
    lr_scheduler = ReduceLROnPlateau(optimizer, mode='max', factor=0.1, patience=3, verbose=True)

    # הגדרת EarlyStopping ומשתנה לשמירת המודל הטוב ביותר (עודכן ל-mAP)
    early_stopping  = EarlyStopping(patience=5, min_delta=0.001, mode='max')
    NUM_EPOCHS      = 30 # מספר אפוקס מקסימלי לאימון
    best_val_mAP    = float('-inf') # אתחול mAP הטוב ביותר כמינוס אינסוף (רוצים למקסם אותו)
    best_model_path = "best_fasterrcnn_model.pth" # נתיב לשמירת המודל הטוב ביותר

    # לולאת אימון + ולידציה
    for epoch in range(1, NUM_EPOCHS + 1):
        print(f"\n---- אפוק {epoch} ----")
        train_loss = train_one_epoch(model, optimizer, data_loader_train, DEVICE, epoch)
        print(f"איבוד באימון ממוצע (Epoch {epoch}): {train_loss:.4f}")

        # ביצוע הערכה וקבלת mAP בוולידציה
        val_mAP = evaluate(model, data_loader_val, DEVICE, coco_evaluator)
        print(f"mAP בוולידציה (Epoch {epoch}): {val_mAP:.4f}")

        # עדכון לומד קצב הלמידה בהתבסס על mAP הוולידציה
        lr_scheduler.step(val_mAP)

        # שמירת המודל הטוב ביותר (רק אם ה-mAP השתפר)
        if val_mAP > best_val_mAP:
            print(f"שומר מודל טוב יותר! mAP ולידציה עלה מ-{best_val_mAP:.4f} ל-{val_mAP:.4f}")
            best_val_mAP = val_mAP
            torch.save(model.state_dict(), best_model_path)

        # בדיקת תנאי עצירה מוקדמת
        if early_stopping.step(val_mAP):
            print(f"עצירת אימון מוקדמת באפוק {epoch} עקב חוסר שיפור ב-mAP ולידציה")
            break

    print(f"\nאימון הושלם. המודל הטוב ביותר נשמר ב-{best_model_path}")

    # --- שלב הסקה על תמונה מסט הוולידציה (דוגמה אקראית) ---
    print("\n--- מבצע הסקה על תמונת בדיקה אקראית מסט הוולידציה ---")

    # טוען את המודל המאומן הטוב ביותר
    model.load_state_dict(torch.load(best_model_path, map_location=DEVICE))
    model.to(DEVICE)
    model.eval() # מעביר למצב הערכה

    def visualize_and_infer(model, image_path, class_map, score_threshold=0.7):
        """
        מבצע הסקה על תמונה בודדת ומציג את הזיהויים.
        """
        rev_class_map = {v: k for k, v in class_map.items()} # מפה הפוכה לתוויות
        img_orig = Image.open(image_path).convert("RGB")
        transform = torchvision.transforms.Compose([torchvision.transforms.ToTensor()])
        img_tensor = transform(img_orig).to(DEVICE)

        with torch.no_grad(): # אין צורך בחישוב גרדיאנטים בהסקה
            prediction = model([img_tensor])

        # חילוץ תוצאות: תיבות, תוויות וציוני ביטחון
        boxes  = prediction[0]['boxes'].cpu().numpy()
        labels = prediction[0]['labels'].cpu().numpy()
        scores = prediction[0]['scores'].cpu().numpy()

        draw = ImageDraw.Draw(img_orig)
        # ניסיון לטעון גופן נפוץ בקולאב, או גופן ברירת מחדל
        font_path = "/usr/share/fonts/truetype/liberation/LiberationSans-Regular.ttf"
        try:
            if os.path.exists(font_path):
                font = ImageFont.truetype(font_path, 20)
            else:
                font = ImageFont.load_default()
        except Exception:
            font = ImageFont.load_default()

        detected_objects = []
        for box, label, score in zip(boxes, labels, scores):
            # מציג רק זיהויים מעל סף הציון הנתון ולא רקע
            if score > score_threshold and rev_class_map.get(label) != '__background__':
                x_min, y_min, x_max, y_max = box.astype(int)
                label_name = rev_class_map.get(label, "Unknown")

                draw.rectangle([(x_min, y_min), (x_max, y_max)], outline="red", width=3)
                text_to_display = f"{label_name}: {score:.2f}"

                # חישוב גודל הטקסט כדי לצייר רקע מתאים
                try: # לגרסאות חדשות של Pillow
                    bbox_text = draw.textbbox((x_min, y_min), text_to_display, font=font)
                    text_width  = bbox_text[2] - bbox_text[0]
                    text_height = bbox_text[3] - bbox_text[1]
                except AttributeError: # לגרסאות ישנות יותר של Pillow
                    text_width, text_height = draw.textsize(text_to_display, font=font)

                # הוספת מרווח קטן לתיבת הטקסט
                draw.rectangle([(x_min, y_min), (x_min + text_width + 4, y_min + text_height + 4)], fill="white")
                draw.text((x_min + 2, y_min + 2), text_to_display, fill="black", font=font)

                detected_objects.append({
                    "label": label_name,
                    "score": float(score),
                    "box": [int(x_min), int(y_min), int(x_max), int(y_max)]
                })

        output_filename   = os.path.basename(image_path).split('.')[0] + "_detected.jpg"
        output_image_path = os.path.join("/content/", output_filename) # שומר בתיקיית ה-root של קולאב
        img_orig.save(output_image_path)
        print(f"התמונה עם הזיהויים נשמרה ב: {output_image_path}")

        return detected_objects

    # בודקים אם יש תמונות בסט הוולידציה לביצוע הסקה לדוגמה
    if dataset_val.image_files_with_xmls:
        random_idx = np.random.randint(0, len(dataset_val.image_files_with_xmls))
        # קבלת נתיב התמונה מתוך הרשימה שנשמרה ב-dataset_val
        example_image_path = dataset_val.image_files_with_xmls[random_idx][0]

        if example_image_path:
            print(f"מבצע הסקה על התמונה: {example_image_path}")
            # סף הציון עבור ההסקה הסופית - ניתן לשנות
            detections = visualize_and_infer(model, example_image_path, CLASS_MAP, score_threshold=0.7)
            if detections:
                print("אובייקטים שזוהו:")
                for det in detections:
                    print(f"    מחלקה: {det['label']}, ציון: {det['score']:.2f}, תיבה: {det['box']}")
            else:
                print("לא זוהו אובייקטים בסף הציון הנתון.")
        else:
            print("שגיאה: לא נמצאה תמונה לדוגמה בתיקיית הוולידציה.")
    else:
        print("אין תמונות בסט הוולידציה לביצוע הסקה.")

if __name__ == '__main__':
    main()

ModuleNotFoundError: No module named 'torchvision.models.detection.coco_utils'

In [ ]:
import matplotlib.pyplot as plt
import matplotlib.patches as patches

def visualize_predictions(model, dataset, device, num_images=5, score_threshold=0.5):
    model.eval()
    for idx in range(num_images):
        img, target = dataset[idx]
        img_tensor = img.to(device).unsqueeze(0)
        with torch.no_grad():
            output = model(img_tensor)[0]

        img_np = img.permute(1, 2, 0).cpu().numpy()
        plt.figure(figsize=(8, 8))
        plt.imshow(img_np)
        ax = plt.gca()

        # ניבויים
        for box, label, score in zip(output['boxes'], output['labels'], output['scores']):
            if score < score_threshold:
                continue
            xmin, ymin, xmax, ymax = box
            rect = patches.Rectangle((xmin, ymin), xmax - xmin, ymax - ymin,
                                     linewidth=2, edgecolor='r', facecolor='none')
            ax.add_patch(rect)
            class_name = list(CLASS_MAP.keys())[list(CLASS_MAP.values()).index(label.item())]
            ax.text(xmin, ymin - 5, f"{class_name} {score:.2f}", color='white',
                    bbox=dict(facecolor='red', alpha=0.5))

        # תיבות אמת (GT)
        for box, label in zip(target['boxes'], target['labels']):
            xmin, ymin, xmax, ymax = box
            rect = patches.Rectangle((xmin, ymin), xmax - xmin, ymax - ymin,
                                     linewidth=1.5, edgecolor='green', facecolor='none')
            ax.add_patch(rect)
            class_name = list(CLASS_MAP.keys())[list(CLASS_MAP.values()).index(label.item())]
            ax.text(xmin, ymax + 10, f"{class_name} (GT)", color='green',
                    bbox=dict(facecolor='black', alpha=0.3))

        plt.axis('off')
        plt.title(f"Image {idx+1} Predictions (red) and GT (green)")
        plt.show()


In [ ]:
import torch
import torchvision
from torchvision.models.detection.faster_rcnn import FasterRCNN_ResNet50_FPN_Weights
from torchvision.models.detection import FasterRCNN
from torch.utils.data import DataLoader, Dataset
from torch.optim.lr_scheduler import StepLR
import os
import xml.etree.ElementTree as ET
from PIL import Image, ImageDraw, ImageFont
import numpy as np
import warnings
from torchvision import transforms as T
from torchvision.models.detection.coco_utils import get_coco_api_from_dataset
from torchvision.models.detection.coco_eval import CocoEvaluator

#  אזהרות מיותרות מ-Pillow
warnings.filterwarnings("ignore", category=UserWarning, module="PIL")

ROOT_DIR_TRAIN_IMAGES      = r"/content/final_burn_dataset/train/train_images"
ROOT_DIR_TRAIN_ANNOTATIONS = r"/content/final_burn_dataset/train/train_xmls"
ROOT_DIR_TEST_IMAGES       = r"/content/final_burn_dataset/test/test_images"
ROOT_DIR_TEST_ANNOTATIONS  = r"/content/final_burn_dataset/test/test_xmls"  # ישמש כסט ולידציה

DEVICE = torch.device('cuda') if torch.cuda.is_available() else torch.device('cpu')
print(f"משתמש ב- DEVICE: {DEVICE}")

CLASS_MAP = {
    '__background__': 0,
    'degree_1':       1,
    'degree_2':       2,
    'degree_3':       3
}
NUM_CLASSES = len(CLASS_MAP)  # = 4


def get_transform(train):
    if train:
        return T.Compose([
            T.ToTensor(),
            T.RandomHorizontalFlip(0.5),
            T.ColorJitter(brightness=0.2, contrast=0.2, saturation=0.2, hue=0.02),
            T.RandomAdjustSharpness(sharpness_factor=2, p=0.3),
        ])
    else:
        return T.Compose([ T.ToTensor() ])

def collate_fn(batch):
    return tuple(zip(*batch))

class CustomDataset(Dataset):
    def __init__(self, root_dir_images, root_dir_annotations, transforms=None, class_map=None):
        self.root_dir        = root_dir_images
        self.annotations_dir = root_dir_annotations
        self.transforms      = transforms
        self.class_map       = class_map or CLASS_MAP

        self.image_ids = self._load_and_validate_image_ids()
        print(f"בתיקייה {root_dir_images}: סה\"כ תמונות וביאורים נטענו בהצלחה: {len(self.image_ids)}")

    def _load_and_validate_image_ids(self):
        image_ids = []
        xml_files = [f for f in os.listdir(self.annotations_dir) if f.endswith('.xml')]

        for xml_file_name in xml_files:
            base_name = os.path.splitext(xml_file_name)[0]

            found_img_path = None
            for ext in ['.jpg', '.jpeg', '.png', '.bmp', '.tiff']:
                temp_path = os.path.join(self.root_dir, base_name + ext)
                if os.path.exists(temp_path):
                    found_img_path = temp_path
                    break

            if found_img_path is None:
                print(f"אזהרה: תמונה {base_name} לא נמצאה עבור ביאור {xml_file_name}. מדלג על זוג זה.")
                continue

            image_ids.append(base_name)
        return image_ids

    def __getitem__(self, idx):
        img_id_base = self.image_ids[idx]

        img_path = None
        for ext in ['.jpg', '.jpeg', '.png', '.bmp', '.tiff']:
            temp_path = os.path.join(self.root_dir, img_id_base + ext)
            if os.path.exists(temp_path):
                img_path = temp_path
                break

        if img_path is None:
            raise FileNotFoundError(f"קובץ תמונה לא נמצא עבור {img_id_base} עם אף סיומת נפוצה.")

        img = Image.open(img_path).convert("RGB")

        xml_path = os.path.join(self.annotations_dir, img_id_base + '.xml')
        tree = ET.parse(xml_path)
        root = tree.getroot()

        boxes  = []
        labels = []
        for obj in root.findall('object'):
            name = obj.find('name').text
            if name not in self.class_map:
                print(f"אזהרה: מחלקה '{name}' לא ממופה ב-CLASS_MAP עבור תמונה {img_id_base}. מדלג על אובייקט זה.")
                continue

            label_id = self.class_map[name]
            bndbox   = obj.find('bndbox')
            xmin = float(bndbox.find('xmin').text)
            ymin = float(bndbox.find('ymin').text)
            xmax = float(bndbox.find('xmax').text)
            ymax = float(bndbox.find('ymax').text)

            boxes.append([xmin, ymin, xmax, ymax])
            labels.append(label_id)

        if len(boxes) == 0:
            boxes  = torch.zeros((0, 4), dtype=torch.float32)
            labels = torch.zeros((0,), dtype=torch.int64)
        else:
            boxes  = torch.as_tensor(boxes, dtype=torch.float32)
            labels = torch.as_tensor(labels, dtype=torch.int64)

        target = {
            "boxes":    boxes,
            "labels":   labels,
            "image_id": torch.tensor([idx]),
            "area":     (boxes[:, 3] - boxes[:, 1]) * (boxes[:, 2] - boxes[:, 0])
                        if len(boxes) > 0
                        else torch.zeros((0,), dtype=torch.float32),
            "iscrowd":  torch.zeros((len(boxes),), dtype=torch.int64)
        }

        if self.transforms is not None:
            img = self.transforms(img)

        return img, target

    def __len__(self):
        return len(self.image_ids)

def create_fasterrcnn_model(num_classes):
    model = torchvision.models.detection.fasterrcnn_resnet50_fpn(
        weights=FasterRCNN_ResNet50_FPN_Weights.COCO_V1
    )
    in_features = model.roi_heads.box_predictor.cls_score.in_features
    model.roi_heads.box_predictor = torchvision.models.detection.faster_rcnn.FastRCNNPredictor(
        in_features, num_classes
    )
    return model

def train_one_epoch(model, optimizer, data_loader, device, epoch):
    model.train()
    total_loss = 0
    for i, (images, targets) in enumerate(data_loader):
        images  = list(image.to(device) for image in images)
        targets = [{k: v.to(device) for k, v in t.items()} for t in targets]

        loss_dict = model(images, targets)
        losses    = sum(loss for loss in loss_dict.values())

        optimizer.zero_grad()
        losses.backward()
        optimizer.step()

        total_loss += losses.item()
        if i % 10 == 0:
            print(f"Epoch: {epoch}, Iteration: {i}/{len(data_loader)}, Loss: {losses.item():.4f}")
    return total_loss / len(data_loader)

def evaluate(model, data_loader, device):
    #  המודל למצב אימון באופן זמני כדי לקבל את מילון ההפסדים
    model.train()
    print("מבצע הערכה על סט הבדיקה...")
    eval_losses = []
    with torch.no_grad(): # לא צריך גרדיאנטים להערכה
        for i, (images, targets) in enumerate(data_loader):
            images  = list(image.to(device) for image in images)
            targets = [{k: v.to(device) for k, v in t.items()} for t in targets]

            loss_dict = model(images, targets)
            losses    = sum(loss for loss in loss_dict.values())
            eval_losses.append(losses.item())

            if i % 50 == 0:
                print(f"Eval Batch {i}/{len(data_loader)}, Loss: {losses.item():.4f}")

    avg_eval_loss = np.mean(eval_losses) if eval_losses else 0
    print(f"הערכה בסיסית הושלמה. ממוצע הפסד ולידציה: {avg_eval_loss:.4f}")

    #  לאחר חישוב הפסד הוולידציה
    model.eval()
    return avg_eval_loss

class EarlyStopping:
    def __init__(self, patience=5, min_delta=0):
        self.patience   = patience
        self.min_delta  = min_delta
        self.counter    = 0
        self.best_loss  = None
        self.early_stop = False

    def step(self, val_loss):
        if self.best_loss is None:
            self.best_loss = val_loss
            return False

        if val_loss < self.best_loss - self.min_delta:
            self.best_loss = val_loss
            self.counter   = 0
            return False
        else:
            self.counter += 1
            print(f"EarlyStopping counter: {self.counter} מתוך {self.patience}")
            if self.counter >= self.patience:
                self.early_stop = True
                return True
            return False

def main():
    print("טוען נתוני אימון")
    dataset_train = CustomDataset(
        root_dir_images=ROOT_DIR_TRAIN_IMAGES,
        root_dir_annotations=ROOT_DIR_TRAIN_ANNOTATIONS,
        transforms=get_transform(train=True),
        class_map=CLASS_MAP
    )

    print("טוען נתוני בדיקה")
    dataset_val = CustomDataset(
        root_dir_images=ROOT_DIR_TEST_IMAGES,
        root_dir_annotations=ROOT_DIR_TEST_ANNOTATIONS,
        transforms=get_transform(train=False),
        class_map=CLASS_MAP
    )

    data_loader_train = DataLoader(
        dataset_train, batch_size=4, shuffle=True,
        num_workers=2, collate_fn=collate_fn
    )
    data_loader_val = DataLoader(
        dataset_val, batch_size=4, shuffle=False,
        num_workers=2, collate_fn=collate_fn
    )

    print("יוצר מודל")
    model = create_fasterrcnn_model(NUM_CLASSES)
    model.to(DEVICE)

    # הגדרת אופטימייזר ולומד קצב
    params    = [p for p in model.parameters() if p.requires_grad]
    optimizer = torch.optim.SGD(params, lr=0.005, momentum=0.9, weight_decay=0.0005)
    lr_scheduler = StepLR(optimizer, step_size=3, gamma=0.1)

    early_stopping  = EarlyStopping(patience=5, min_delta=0.001)
    NUM_EPOCHS      = 30
    best_val_loss   = float('inf')
    best_model_path = "best_fasterrcnn_model.pth"

    # לולאת אימון + ולידציה
    for epoch in range(1, NUM_EPOCHS + 1):
        print(f"\n---- אפוק {epoch} ----")
        train_loss = train_one_epoch(model, optimizer, data_loader_train, DEVICE, epoch)
        print(f"איבוד באימון (Epoch {epoch}): {train_loss:.4f}")

        val_loss = evaluate(model, data_loader_val, DEVICE)
        print(f"איבוד ולידציה (Epoch {epoch}): {val_loss:.4f}")

        lr_scheduler.step()

        # שמירת המודל הטוב ביותר לפי ולידציה
        if val_loss < best_val_loss:
            print(f"שומר מודל טוב יותר! איבוד ולידציה ירד מ-{best_val_loss:.4f} ל-{val_loss:.4f}")
            best_val_loss = val_loss
            torch.save(model.state_dict(), best_model_path)

        # בדיקת EarlyStopping
        if early_stopping.step(val_loss):
            print(f"עצירת אימון מוקדמת באפוק {epoch} עקב חוסר שיפור באיבוד ולידציה")
            break

    print(f"\nאימון הושלם. המודל הטוב ביותר נשמר ב-{best_model_path}")

    # שלב הסקה על תמונה מסט הוולידציה (דוגמה אקראית)
    print("\n--- מבצע הסקה על תמונת בדיקה אקראית מסט הוולידציה ---")
    model.load_state_dict(torch.load(best_model_path, map_location=DEVICE))
    model.to(DEVICE)
    model.eval()

    def visualize_and_infer(model, image_path, class_map, score_threshold=0.7):
        rev_class_map = {v: k for k, v in class_map.items()}
        img_orig = Image.open(image_path).convert("RGB")
        transform = torchvision.transforms.Compose([torchvision.transforms.ToTensor()])
        img_tensor = transform(img_orig).to(DEVICE)

        with torch.no_grad():
            prediction = model([img_tensor])

        boxes  = prediction[0]['boxes'].cpu().numpy()
        labels = prediction[0]['labels'].cpu().numpy()
        scores = prediction[0]['scores'].cpu().numpy()

        draw = ImageDraw.Draw(img_orig)
        font_path = "/usr/share/fonts/truetype/liberation/LiberationSans-Regular.ttf"
        try:
            if not os.path.exists(font_path):
                font = ImageFont.load_default()
            else:
                font = ImageFont.truetype(font_path, 20)
        except Exception:
            font = ImageFont.load_default()

        detected_objects = []
        for box, label, score in zip(boxes, labels, scores):
            if score > score_threshold and rev_class_map.get(label) != '__background__':
                x_min, y_min, x_max, y_max = box.astype(int)
                label_name = rev_class_map.get(label, "Unknown")

                draw.rectangle([(x_min, y_min), (x_max, y_max)], outline="red", width=3)
                text_to_display = f"{label_name}: {score:.2f}"
                bbox_text = draw.textbbox((x_min, y_min), text_to_display, font=font)
                text_width  = bbox_text[2] - bbox_text[0]
                text_height = bbox_text[3] - bbox_text[1]
                draw.rectangle([(x_min, y_min), (x_min + text_width, y_min + text_height)], fill="white")
                draw.text((x_min, y_min), text_to_display, fill="black", font=font)

                detected_objects.append({
                    "label": label_name,
                    "score": float(score),
                    "box": [int(x_min), int(y_min), int(x_max), int(y_max)]
                })

        output_filename   = os.path.basename(image_path).split('.')[0] + "_detected.jpg"
        output_image_path = os.path.join("/content/", output_filename)
        img_orig.save(output_image_path)
        print(f"התמונה עם הזיהויים נשמרה ב: {output_image_path}")

        return detected_objects

    # בודקים אם יש תמונות בסט הוולידציה
    if dataset_val.image_ids:
        # בוחר תמונה אקראית במקום הראשונה
        random_idx = np.random.randint(0, len(dataset_val.image_ids))
        example_image_base_name = dataset_val.image_ids[random_idx]
        example_image_path = None
        for ext in ['.jpg', '.jpeg', '.png', '.bmp', '.tiff']:
            temp_path = os.path.join(ROOT_DIR_TEST_IMAGES, example_image_base_name + ext)
            if os.path.exists(temp_path):
                example_image_path = temp_path
                break

        if example_image_path:
            print(f"מבצע הסקה על התמונה: {example_image_path}")
            detections = visualize_and_infer(model, example_image_path, CLASS_MAP, score_threshold=0.7)
            if detections:
                print("אובייקטים שזוהו:")
                for det in detections:
                    print(f"    מחלקה: {det['label']}, ציון: {det['score']:.2f}, תיבה: {det['box']}")
            else:
                print("לא זוהו אובייקטים בסף הציון הנתון.")
        else:
            print("שגיאה: לא נמצאה תמונה לדוגמה בתיקיית הוולידציה.")
    else:
        print("אין תמונות בסט הוולידציה לביצוע הסקה.")

if __name__ == '__main__':
    main()

משתמש ב- DEVICE: cuda
טוען נתוני אימון...
בתיקייה /content/final_burn_dataset/train/train_images: סה"כ תמונות וביאורים נטענו בהצלחה: 981
טוען נתוני בדיקה...
בתיקייה /content/final_burn_dataset/test/test_images: סה"כ תמונות וביאורים נטענו בהצלחה: 246
יוצר מודל...

---- אפוק 1 ----
Epoch: 1, Iteration: 0/246, Loss: 2.0309
Epoch: 1, Iteration: 10/246, Loss: 0.2552
Epoch: 1, Iteration: 20/246, Loss: 0.3470
Epoch: 1, Iteration: 30/246, Loss: 0.6059
Epoch: 1, Iteration: 40/246, Loss: 0.2361
Epoch: 1, Iteration: 50/246, Loss: 0.2754
Epoch: 1, Iteration: 60/246, Loss: 0.3194
Epoch: 1, Iteration: 70/246, Loss: 0.3210
Epoch: 1, Iteration: 80/246, Loss: 0.1892
Epoch: 1, Iteration: 90/246, Loss: 0.3121
Epoch: 1, Iteration: 100/246, Loss: 0.4975
Epoch: 1, Iteration: 110/246, Loss: 0.4148
Epoch: 1, Iteration: 120/246, Loss: 0.3224
Epoch: 1, Iteration: 130/246, Loss: 0.2594
Epoch: 1, Iteration: 140/246, Loss: 0.4143
Epoch: 1, Iteration: 150/246, Loss: 0.2326
Epoch: 1, Iteration: 160/246, Loss: 0.3330

In [ ]:
from google.colab import files
files.download('/content/best_fasterrcnn_model.pth')


<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

המודל הורד בהצלחה למחשב שלך.
